<a href="https://colab.research.google.com/github/Gurinderdev/GG_Clean/blob/master/Convolution_MNIST_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# get images from mnist
# create noise 
# develop gan structure 


In [0]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets 
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# # Set random seem for reproducibility
# manualSeed = 999
# #manualSeed = random.randint(1, 10000) # use if you want new results
# print("Random Seed: ", manualSeed)
# random.seed(manualSeed)
# torch.manual_seed(manualSeed)

In [0]:
batch_size = 15

d_learning_rate = 0.0002
g_learning_rate = 0.0002

epoch = 100


In [0]:
# get images from mnist

transform =  torchvision.transforms.Compose([transforms.ToTensor() ,
                                      transforms.Normalize([0.5] , [0.5]) ])
    
dataset = torchvision.datasets.MNIST('mnist' , download = True , train = True , transform= transform )

trainloader = torch.utils.data.DataLoader(dataset , shuffle = True , batch_size = batch_size)

0it [00:00, ?it/s]

9920512it [00:02, 4471498.24it/s]                             


Extracting mnist/MNIST/raw/train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

32768it [00:00, 66976.28it/s]                            
0it [00:00, ?it/s]

Extracting mnist/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:01, 1098433.47it/s]                            
0it [00:00, ?it/s]

Extracting mnist/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 25033.74it/s]            

Extracting mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
# get fake data as a 100 dimension gausian 
def fake():
  return torch.rand(1,100,1,1)

In [0]:

class Generator(nn.Module):
    def __init__(self  ):
        super(Generator, self).__init__()
        self.features = nn.Sequential(
                
           nn.ConvTranspose2d(in_channels= 100 , out_channels= 28*4 , kernel_size= (2,2) , stride= (1,1) )
         , nn.BatchNorm2d(28*4)
         , nn.ReLU() 
         , nn.ConvTranspose2d(in_channels= 28*4 , out_channels= 28*2 , kernel_size= (2,2) , stride = (1,1))
         , nn.BatchNorm2d(28*2)
         , nn.ReLU()  
         , nn.ConvTranspose2d(in_channels= 28*2 , out_channels= 28 , kernel_size= (2,2) , stride = (1,1))
         , nn.BatchNorm2d(28)
         , nn.ReLU() 
          , nn.ConvTranspose2d(in_channels= 28 , out_channels= 1, kernel_size= (2,2) , stride = (1,1))
         , nn.ReLU()
        )
                  
        
        
    def forward(self, x):
         print("first/ input  {}".format(x.size()))
         x = self.features(x)
         print("output sise {}".format(x.size()))
         x = x.view(1*28*28)
          
          
         return x
    

In [0]:
g = Generator()
g(fake())

first/ input  torch.Size([1, 100, 1, 1])
output sise torch.Size([1, 1, 5, 5])


RuntimeError: ignored

In [0]:
def deconv(in_channels, out_channels, kernel_size, stride=2, padding=1, batch_norm=True, bias=False):
    """Creates a transposed-convolutional layer, with optional batch normalization.
    """
    layers = []
    layers.append(nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=bias))
    if batch_norm:
        layers.append(nn.BatchNorm2d(out_channels))
    return nn.Sequential(*layers)

In [0]:
class Generator_test(nn.Module):
    def __init__(self, ngpu):
        super(Generator_test, self).__init__()
        self.ngpu = ngpu
        # Fill this in

        self.convT1 = deconv(nz, ngf*8, 4, 1, 0)
        self.convT2 = deconv(ngf*8, ngf*4, 4, 2, 1)
        self.convT3 = deconv(ngf*4, ngf*2, 4, 2, 1)
        self.convT4 = deconv(ngf*2, ngf, 4, 2, 1)
        self.output = deconv(ngf, nc, 4, 2, 1, batch_norm=False)

    def forward(self, input):
        out = F.relu(self.convT1(input))
        out = F.relu(self.convT2(out))
        out = F.relu(self.convT3(out))
        out = F.relu(self.convT4(out))
        

        # What non-linearity on the output layer
        #return ?(self.output(out))
        out = torch.tanh(self.output(out))  
        print(out.size())
        return
      
# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

ngpu = 1


In [0]:
g_test = Generator_test(ngpu= 1)
g_test(fake())

torch.Size([1, 3, 64, 64])


first torch.Size([1, 100, 1, 1])
torch.Size([1, 1, 5, 5])


RuntimeError: ignored

In [0]:

class Generator_1(nn.Module):
    def __init__(self , ngpu):
        super(Generator_1, self).__init__()
                   
        self.conv1 =     nn.ConvTranspose2d(in_channels= 1 , out_channels= 28*4 , kernel_size= (2,2) , stride= (2,2) )
        self.con1_b = nn.BatchNorm2d(28*4)
        self.conv2 = nn.ConvTranspose2d(in_channels= 28*4 , out_channels= 28*2 , kernel_size= (2,2) , stride = (2,2))
        self.conv2_b = nn.BatchNorm2d(28*2)
        self.conv3 =  nn.ConvTranspose2d(in_channels= 28*2 , out_channels= 28 , kernel_size= (2,2) , stride = (2,2))
        
         , nn.BatchNorm2d(28*4)
         , nn.ReLU() 
         , 
         , 
         , nn.ReLU()  
         ,
         , nn.ReLU() )
                  
        
        
    def forward(self, x):
         x = nn.ReLU(self.conv1(x))

         x = nn.ReLU(self.conv2(x))
          
          x = 
         x = self.conv2(x)
          
         x = x.view(1,28,28)
          
         return x
    

In [0]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        
        self.para
        
        self.input = nn.Linear(input_size, hidden_size)
        self.hidden = nn.Linear(hidden_size, hidden_size)
        self.hidden_2 = nn.Linear(hidden_size, hidden_2)
        self.output = nn.Linear(hidden_2 , output_size)
        self.nonLinearity = torch.nn.ELU()
        self.nonLinearityOutput = torch.sigmoid

    def forward(self, x):
        x = self.nonLinearity(self.input(x))
        x = self.nonLinearity(self.hidden(x))
        x = self.nonLinearity(self.hidden_2(x))
        x = self.nonLinearityOutput(self.output(x))
        return x 

In [0]:
G = Generator(input_size= 100  , hidden_size= 400 , output_size = 784)
D = Discriminator (input_size= 784 , hidden_size= 400 , hidden_2= 100 ,  output_size= 1 )

In [0]:
## loss function and Optimiser

criterion = nn.BCELoss()

## optimiser

d_optimiser = optim.SGD(D.parameters() , lr = d_learning_rate)
g_optimiser = optim.SGD(G.parameters() , lr = g_learning_rate)

In [0]:
# train on fake data 

def train_D_on_FAKE():
  
  
  decision  = D(G(fake(batch_size = batch_size)))
  error = criterion(decision ,torch.zeros(batch_size , 1) )
  error.backward()
  
def train_D_on_REAL(imaage):
  
  decision = D(image)
  error = criterion( decision , torch.ones(batch_size , 1))
  error.backward()

In [0]:
def train_D_on_FAKE():
  
  
  decision  = D(G(fake(batch_size)))
  error = criterion(decision ,torch.zeros(batch_size , 1) )
  error.backward()

In [0]:
fake(batch_size = batch_size)

tensor([[0.7044, 0.4222, 0.8969,  ..., 0.9084, 0.8911, 0.1770],
        [0.1116, 0.4566, 0.5943,  ..., 0.9101, 0.1282, 0.2288],
        [0.9539, 0.7778, 0.7187,  ..., 0.6141, 0.4401, 0.4325],
        ...,
        [0.1699, 0.6160, 0.6134,  ..., 0.6753, 0.3358, 0.1563],
        [0.4795, 0.9529, 0.8799,  ..., 0.0425, 0.3757, 0.0045],
        [0.4828, 0.8167, 0.5114,  ..., 0.5761, 0.3745, 0.8334]])

In [0]:
def train_G():
  
  generated = G(fake(batch_size= batch_size))
  decision  = D(generated)
  error = criterion(decision ,torch.ones(batch_size , 1) )
  error.backward()
  return error.item() ,generated 
  

In [0]:
# train loop 

for e in range(epoch):
  for i, data in enumerate(trainloader , 0 ):
    
    image, _ = data
    image = image.view(-1 , 28*28*1)
    
    # run through discriminator first
    D.zero_grad()
    train_D_on_REAL(image)
    train_D_on_FAKE()
    d_optimiser.step()
    
    # run throught the generator 
    
    G.zero_grad()
    error , generated = train_G()
    g_optimiser.step()
    
    print("Gen_LOSS : {} , Epoch : {} , i: {}"
          .format(error, e , i ))
    
   
    if i/ 4000 == 0: 
      print( 'GAN produced image for e {} , i {}'.format (e, i ))
      
      to_print = plt.imshow(np.squeeze(generated[0].view(-1,28,28).detach().numpy()) , cmap = 'Greys_r')
      plt.savefig('to_print.png')
      
    
    
    
    

Gen_LOSS : 0.7348880767822266 , Epoch : 0 , i: 0
GAN produced image for e 0 , i 0
Gen_LOSS : 0.7350913286209106 , Epoch : 0 , i: 1
Gen_LOSS : 0.7353112101554871 , Epoch : 0 , i: 2
Gen_LOSS : 0.7352199554443359 , Epoch : 0 , i: 3
Gen_LOSS : 0.7357158064842224 , Epoch : 0 , i: 4
Gen_LOSS : 0.7352651953697205 , Epoch : 0 , i: 5
Gen_LOSS : 0.7352283596992493 , Epoch : 0 , i: 6
Gen_LOSS : 0.7350448369979858 , Epoch : 0 , i: 7
Gen_LOSS : 0.7355920672416687 , Epoch : 0 , i: 8
Gen_LOSS : 0.7353290319442749 , Epoch : 0 , i: 9
Gen_LOSS : 0.7356238961219788 , Epoch : 0 , i: 10
Gen_LOSS : 0.7357958555221558 , Epoch : 0 , i: 11
Gen_LOSS : 0.7356611490249634 , Epoch : 0 , i: 12
Gen_LOSS : 0.7355533838272095 , Epoch : 0 , i: 13
Gen_LOSS : 0.7358049154281616 , Epoch : 0 , i: 14
Gen_LOSS : 0.7358787655830383 , Epoch : 0 , i: 15
Gen_LOSS : 0.7356066107749939 , Epoch : 0 , i: 16
Gen_LOSS : 0.7361561059951782 , Epoch : 0 , i: 17
Gen_LOSS : 0.7358031868934631 , Epoch : 0 , i: 18
Gen_LOSS : 0.73620247840881